<a href="https://colab.research.google.com/github/arjun-2131/cow-breed/blob/master/cow_class_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Convolution2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation,Dropout,Flatten,Dense,BatchNormalization
from keras import backend as K
from keras.preprocessing import image
from keras import applications
from keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
from keras.applications.xception import preprocess_input
import seaborn as sns
sns.set(color_codes=True)

In [0]:
batch_size=10
num_classes=2
epochs=2
img_width,img_height=256,256

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
train_data_dir="/content/drive/My Drive/cow/train"
validation_data_dir="/content/drive/My Drive/cow/valid"
test_data_dir="/content/drive/My Drive/cow/test"

In [0]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

In [11]:
  model=applications.xception.Xception(include_top=False,
                                      weights='imagenet',
                                      input_shape=(img_width,img_height,3))

W0619 06:24:17.588795 140047079634816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0619 06:24:17.647112 140047079634816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 06:24:17.656452 140047079634816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0619 06:24:17.717178 140047079634816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0619 06:24:17.718403 1400470796

83689472/83683744 [==============================] - 3s 0us/step


In [0]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=BatchNormalization()(x)
x=Dense(1024,activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(0.5)(x)
y_pred=Dense(2,activation='softmax')(x)

In [13]:
model_final=Model(inputs=model.input,outputs=y_pred)
model_final.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
for layers in model.layers[:126]:
  layers.trainable=False
for layer in model.layers[126:]:
  layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(
              rescale=1./255,
              shear_range=0.2,
              zoom_range=0.2,
              horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [19]:
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
validation_generator=test_datagen.flow_from_directory(validation_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
test_generator=test_datagen.flow_from_directory(test_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

Found 145 images belonging to 2 classes.
Found 15 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [0]:
model_final.compile(loss='categorical_crossentropy',
                   optimizer='nadam',
                   metrics=['accuracy'])   

In [0]:
top_model_weight_path='cow_Xception.h5'
checkpoint=ModelCheckpoint(filepath=top_model_weight_path,
                          verbose=1,
                            save_best_only=True,
                          monitor='val_acc')
early=EarlyStopping(monitor='val_acc',min_delta=0,patience=5,verbose=1,mode='auto')

history=model_final.fit_generator(
              train_generator,
                steps_per_epoch=145//batch_size,
              epochs=epochs,
                validation_data=validation_generator,
                validation_steps=15//batch_size,
                callbacks=[checkpoint,early])

Epoch 1/2
13/14 [==========================>...] - ETA: 26s - loss: 0.2200 - acc: 0.9498